In [1]:
#библиотеки, необходимые для формирования запросов сайту и получение ответов
#и поиска в полученной с сайта информации необхоимых для проекта данных
from bs4 import BeautifulSoup
import requests as req
import lxml.html as html
import urllib3

import time #библиотека для установки паузы между запросами к сайту

#общие библиотеки
import pandas as pd
from IPython.display import Image, HTML
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#для работы с файлами
import shutil

In [2]:
#формируем header для включения его в запросы, так как сайты могут блокировать запросы без него
header = {'user-agent': 'Edg/95.0.1020.53'}
urllib3.disable_warnings()

In [123]:
#этот цикл запускаем для каждого из 10-ти класса автомобилей (здесь стоит для примера volkswagen)
#в цикле пробегаемся запросами по ссылкам на отдельные авто (обычно 25 ссылок на авто на 1 страницу)
#далее по тегам 'div' и классам с картинками вылавливаем из супа ссылки, содержащие фотографии авто
#для сайта авто.ру и дром.ру свои классы и ссылки
images = []

for page_number in range(1,25):
    
    #html_page = req.get('https://auto.ru/cars/volkswagen/passat/all/?page='+str(page_number),
    #                    headers=header, timeout=50, verify=False)
    html_page = req.get('https://auto.drom.ru/volkswagen/passat/page'+str(page_number)+'/?ph=1',
                        headers=header, timeout=50, verify=False)
    
    soup = BeautifulSoup(html_page.content, 'html.parser')
    #cars = soup.findAll('div', class_="ListingItem")
    cars = soup.findAll('div', class_="css-1f4se8o e1e9ee560")

    #for i in range(len(cars)):
    #    if len(cars[i].findAll('img'))==1:
    #        images.append(cars[i].findAll('img')[0].attrs['alt data-src'])
    
    for i in range(len(cars)):
        images.append(cars[i].findAll('img')[0].attrs['data-src'])
    
    
    time.sleep(3) #ставим на 3 секунды задержки между запросами, чтобы не забанили

In [124]:
#проверка количества выловленных ссылок с фото авто
len(images)

480

In [125]:
#в этом цикле пробегаемся по списку собранных ссылок с авто и забираем файл с фото, загрузив его в свою папку
#при этом не забываем для каждого класса от 1 до 10 менять название папки на цифру класса

for i in range(len(images)):
    #url_base = "https:" #Original website
    #url_ext = images[i] #The extension you pulled earlier
    #full_url = url_base + url_ext #Combining first 2 variables to create       a complete URL
    full_url = images[i]
    r = req.get(full_url, stream=True) #Get request on full_url
    if r.status_code == 200:                     #200 status code = OK
        with open('C:/Users/Desktop/auto_pictures/train_additional/8/car'+str(i+8500)+'.jpg', 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)